This code below is not necessary. Its for the categoreies, which we did not need in the end.


In [7]:
import pandas as pd
import numpy as np
from pathlib import Path

# --- Paths ---
input_path = Path("/workspaces/bakery_prediction/0_DataPreparation/Processed/03_ji_featured_data.csv")
output_path = Path("/workspaces/bakery_prediction/0_DataPreparation/Processed/04_ls_weather_categorised.csv")

# --- Load CSV ---
df = pd.read_csv(input_path, parse_dates=["Datum"])

# Ensure the column is numeric
df["Wettercode"] = pd.to_numeric(df["Wettercode"], errors="coerce")

# --- Mapping function ---
def map_weather_code(code):
    if np.isnan(code):
        return np.nan
    c = int(code)
    if 0 <= c <= 3:
        return "sunny"
    elif 4 <= c <= 49:
        return "cloudy"
    elif 50 <= c <= 94:
        return "rainy"
    elif 95 <= c <= 99:
        return "thunderstorm"
    else:
        return np.nan

# Replace numeric codes with text labels
df["Wettercode"] = df["Wettercode"].apply(map_weather_code)

# --- Save the new table ---
df.to_csv(output_path, index=False)




' this is the code for the weather variables (1 or 0)

In [8]:
import pandas as pd
from pathlib import Path

# --- Paths ---
input_path = Path("/workspaces/bakery_prediction/0_DataPreparation/Processed/04_ls_weather_categorised.csv")
output_path = Path("/workspaces/bakery_prediction/0_DataPreparation/Processed/05_ls_weather_variables.csv")

# --- Load data ---
df = pd.read_csv(input_path, parse_dates=["Datum"])

# --- Create 0/1 columns manually ---
df["sunny"] = (df["Wettercode"] == "sunny").astype(int)
df["cloudy"] = (df["Wettercode"] == "cloudy").astype(int)
df["rainy"] = (df["Wettercode"] == "rainy").astype(int)
df["thunderstorm"] = (df["Wettercode"] == "thunderstorm").astype(int)

#delete weathercategory column
df = df.drop(columns=["Wettercode"])

# --- Save new dataset ---
df.to_csv(output_path, index=False)





